In [2]:
import astropy.io.fits as fits
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter
import pandas as pd
import shutil
from function import *

In [3]:
import sys
print(sys.executable)

/bin/python3


##  sky画像の作成  
makedark.ipynbまでで作成した関数により、1Expあたりのダークまでは得られた  
次にobsfileを指定したとき、obsfileのQ_CHEBを取得して、1Expあたりのdarkに掛け算する。それをobsfileから差し引くことでskyとする  
差し引くとき、次元が異なるが、できるかは試してみること

In [4]:
#実験用ファイル
obsfile = r'/mnt/e/2003/COMA00042689.fits'

In [5]:
obsdata = readdata(obsfile)
Q_CHEB = readheader(obsfile)["Q_CHEB"]


In [6]:
make1Expdark(obsfile)

/mnt/e/2003/COMA00042689.fits


array([[[[-2391.37807377, -2407.23258197, -2398.6977459 , ...,
          -2226.99282787, -2253.40061475, -2241.94979508],
         [-2391.75819672, -2416.39651639, -2415.99180328, ...,
          -2226.04098361, -2241.65163934, -2220.98258197],
         [-2386.46413934, -2419.08606557, -2415.38831967, ...,
          -2228.96106557, -2249.20491803, -2227.7817623 ],
         ...,
         [-2481.49282787, -2520.18340164, -2519.91393443, ...,
          -2339.46721311, -2363.26536885, -2341.75614754],
         [-2484.88831967, -2528.57172131, -2524.79918033, ...,
          -2351.78381148, -2368.2807377 , -2359.59221311],
         [-2501.14651639, -2538.14856557, -2527.43545082, ...,
          -2358.83913934, -2370.6875    , -2361.0204918 ]],

        [[-2391.37807377, -2407.23258197, -2398.6977459 , ...,
          -2226.99282787, -2253.40061475, -2241.94979508],
         [-2391.75819672, -2416.39651639, -2415.99180328, ...,
          -2226.04098361, -2241.65163934, -2220.98258197],
        

## sky画像を作成後、Flatで割るところまで  


In [8]:
def make_obj(obsfile):
    # dark_1を取得
    dark_1 = make1Expdark(obsfile)
    # Q_CHEB倍する
    dark_CHEB = dark_1 * int(Q_CHEB)
    #obsdata
    obsdata=readdata(obsfile)
    #skydata作成
    skydata = obsdata - dark_CHEB
    #q_bsepによりposiとnegaにわける
    #Addモードの場合だけでやってみる。
    skydata_p = q_bsep_posi(skydata)
    skydata_n = q_bsep_nega(skydata)

    # z方向平均
    sky_pa = mean_z(skydata_p)
    sky_na = mean_z(skydata_n)

    # gaussian　平均
    sky_paG = gaussfilter(sky_pa)
    sky_naG = gaussfilter(sky_na)

    # Flatの作成
    sky_paF = sky_pa / sky_paG
    sky_naF = sky_na / sky_naG

    comq_obs = readdata(obsfile.replace("COMA", "COMQ"))
    obj_obs = q_subch(comq_obs)
    #Flatで割る
    obj_obs_datP0 = obj_obs / sky_naF
    obj_obs_datN0 = obj_obs / sky_paF

    return obj_obs_datP0, obj_obs_datN0



In [13]:
obsfile = r'/mnt/e/2003/COMA00042689.fits'
obsfile[-10:-5]

'42689'

In [11]:
obj_datP0 = make_obj(obsfile)[0]
obj_datN0 = make_obj(obsfile)[1]

/mnt/e/2003/COMA00042689.fits
/mnt/e/2003/COMA00042689.fits


積分時間で割る
Q_1FRAMEの時間に対して、

In [14]:
outputfolder = r'/mnt/e/2003/out_obj/obj_'
makefits(obj_datP0, outputfolder+obsfile[-10:-5]+"_datP0.fits")
makefits(obj_datN0, outputfolder+obsfile[-10:-5]+"_datN0.fits")


In [19]:
def makefits2(dataname, filename, headerfile):
    hdu = fits.PrimaryHDU(data=dataname,header=readheader(headerfile))
    fits.HDUList([hdu]).writeto(str(filename), overwrite=True)

In [20]:
makefits2(obj_datP0, outputfolder+obsfile[-10:-5]+"_datP0_per1sec.fits", obsfile)


In [22]:
fits.getheader(obsfile)
hdu = fits.PrimaryHDU(data=obj_datP0, header = fits.getheader(obsfile))
fits.HDUList([hdu]).writeto("objdaP0.fits", overwrite=True)

In [15]:
obsfile

'/mnt/e/2003/COMA00042689.fits'

In [18]:
comq_obs = obsfile.replace("COMA", "COMQ")
comq_header = readheader(comq_obs)
comq_header

SIMPLE  =                    T / Standard FITS format                           
BITPIX  =                   32 / # of bits per pixel                            
NAXIS   =                    4 / # of axis in data                              
NAXIS1  =                  320 / # of pixels/row                                
NAXIS2  =                  240 / # of rows                                      
NAXIS3  =                    1 / # of frames                                    
NAXIS4  =                    1 / # of detectors                                 
EXTEND  =                    F / ASCII Extension Table                          
COMMENT = ' +++++++++++++++++++ SUBARU COMMON'                                  
COMMENT = ' ------------------------- About This DATA'                          
FRAMEID = 'COMA00042689'       / Data Serial Num of COMICS A-sequence           
EXP-ID  = 'COME00042689'       / Exporsure ID                                   
OBS-ALOC= 'Observation '    

In [17]:
obsheader = readheader(obsfile)
obsheader

SIMPLE  =                    T / Standard FITS format                           
BITPIX  =                   32 / # of bits per pixel                            
NAXIS   =                    4 / of axis in frame                               
NAXIS1  =                  320 / # of pixels/row                                
NAXIS2  =                  240 / # of pixels/row                                
NAXIS3  =                   12 / # of pixels/row                                
NAXIS4  =                    1 / # of pixels/row                                
EXTEND  =                    F / ASCII Extension Table                          
COMMENT   = ' +++++++++++++++++++ SUBARU COMMON'                                
COMMENT   = ' ------------------------- About This DATA'                        
FRAMEID = 'COMA00042689'       /  Data Serial Num of COMICS A-sequence          
EXP-ID  = 'COME00042689'       /  Exporsure ID                                  
OBS-ALOC= 'Observation'     